In [1]:
import pandas as pd
from urllib.request import Request, urlretrieve
from pathlib import Path  
import ssl
import os
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import tensorflow as tf
from art_api import config

In [3]:
def resize(dir, dir_sm):
    '''Resize image to 256x256 (to be implemented: if not already in local disk)
    Args: make sure you specify the relative file path,
    e.g. resize("../raw_data/test", "../raw_data/test_sm")
        dir: source directory
        dir_sm: destination directory
    Returns:
        String output describing number of images resized
    '''
    counter = 0
    for file in os.listdir(dir):
        image = Image.open(os.path.join(dir, file))
        image = image.resize((224, 224), Image.ANTIALIAS)
        if not os.path.exists(dir_sm):
            print(f"Directory does not exist. Creating {dir_sm}.")
            os.makedirs(dir_sm)
        image.save(os.path.join(dir_sm, file))
        counter +=1
    return f"{counter} images resized and rescaled to {dir_sm}"

In [4]:
resize("../raw_data/aws10k", "../raw_data/aws10k_sm")

/tmp/ipykernel_1839/2657577696.py:13: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((224, 224), Image.ANTIALIAS)


Directory does not exist. Creating ../raw_data/aws10k_sm.


'4962 images resized and rescaled to ../raw_data/aws10k_sm'

In [5]:
resize("../raw_data/google", "../raw_data/google_sm")

/tmp/ipykernel_1839/2657577696.py:13: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((224, 224), Image.ANTIALIAS)


Directory does not exist. Creating ../raw_data/google_sm.


'5512 images resized and rescaled to ../raw_data/google_sm'

In [7]:
resize("../raw_data/yourpaintings", "../raw_data/yourpaintings_sm")

/tmp/ipykernel_1839/2657577696.py:13: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((224, 224), Image.ANTIALIAS)


'8255 images resized and rescaled to ../raw_data/yourpaintings_sm'

In [2]:
from art_api import config, data, utils, preproc, trainer
import pandas as pd
from art_api import config, data, utils, preproc
import os
from google.cloud import storage
from IPython.display import Image
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential, layers, models
from sklearn.model_selection import train_test_split
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.applications.resnet import ResNet152, preprocess_input as preproc_resnet
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
import seaborn as sns
from keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow import keras

In [24]:
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))

In [25]:
loss_history = LossHistory()
lrate = LearningRateScheduler(step_decay)
# callbacks_list = [loss_history, lrate]
# history = model.fit(X_train, y_train, 
#    validation_data=(X_test, y_test), 
#    epochs=epochs, 
#    batch_size=batch_size, 
#    callbacks=callbacks_list, 
#    verbose=2)

In [ ]:
'binary_crossentropy'

In [16]:
def load_data():

    imgs = []
    df = data.DataLoader.add_img("../raw_data/google_sm")
    df_sample = preproc.sample(df)
    X, y = utils.load_add(df_sample.head(100))
    
    print(X.shape)
    print(y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)
    print(f"the shape of X_train is {X_train.shape}")
    print(f"the shape of X_val is {X_val.shape}")
    print(f"the shape of X_test is {X_test.shape}")
    print(f"the shape of y_train is {y_train.shape}")
    print(f"the shape of y_val is {y_val.shape}")
    print(f"the shape of y_test is {y_test.shape}")

    print("\n\nMake sure we're using GPU\n\n")
    return X_train, X_val, X_test, y_train, y_val, y_test

In [8]:
def tuner():
    """Add to dataset with images scraped off Google +
    Augment images for training (xxx)
    + Unfree last block
    + Use batch size of 16
    """
    X_train, X_val, X_test, y_train, y_val, y_test = load_data()
    
    try:
        physical_devices = tf.config.experimental.list_physical_devices("GPU")
        if len(physical_devices) > 0:
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
        os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
        print(os.getenv("TF_GPU_ALLOCATOR"))
        print(f"memory usage {tf.config.experimental.get_memory_info('GPU:0')['current'] / 10 ** 9} GB")
    except:
        pass
    
    print("\n\nFine-tune pre-trained ResNet152 model\n\n")
    
    model = load_model("../raw_data/models/resnet152")
    model_name = "tuned_resnet152_existing_test"
    model.summary()
    
    print("\n\nIndex the number of layers in the ResNet152 base\n\n")
    
    model.layers[0].summary()
    print(f"There are {len(model.layers[0].layers)} layers in the base model.")

    # freeze base, with exception of last layer
    model.trainable = True
    set_trainable = False

    for layer in model.layers[0].layers:
        if 'conv5_block3' in layer.name:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    # sanity check on trainable/untrainable params in base
    model.layers[0].summary()
    
    # sanity check on trainable/untrainable params in model
    print(f"Model summary after unfreezing conv5_block3 for training")
    model.summary()

    resnet50_X_train = preproc_resnet(X_train) 
    resnet50_X_val = preproc_resnet(X_val)
    resnet50_X_test = preproc_resnet(X_test)
    
    # Custom Scheduler Function
#     lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,
#                                                               decay_steps=10000,
#                                                               decay_rate=0.9)
        
    # compile and fit the model; during trainining, augment training images, change batch size to 16
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=optimizers.Adam(learning_rate=1e-2),
                  metrics=['accuracy'])
            
    es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 30, 
                   verbose = 1, 
                   restore_best_weights = True)
    
#    callbacks_list = [loss_history, lrate, es]
    history = model.fit(resnet50_X_train, y_train, 
                    validation_data=(resnet50_X_val, y_val), 
                    epochs=150, 
                    batch_size=32, 
                    callbacks=[callbacks_list])
    
    if os.path.exists(f"../raw_data/models/{model_name}/"):
        model.save(f"../raw_data/models/{model_name}")
    else:
        os.makedirs(f"../raw_data/models/{model_name}")
        print(f"The directory for {model_name} is created.")
  
    model.save(f"../raw_data/models/{model_name}")
    print(f"Model {model_name} saved.")
    return model, model_name, history, resnet50_X_train, resnet50_X_val, resnet50_X_test

In [17]:
X_train, X_val, X_test, y_train, y_val, y_test = load_data()

758 sampled per aeroplane
758 sampled per bird
758 sampled per boat
758 sampled per chair
758 sampled per cow
758 sampled per diningtable
758 sampled per dog
758 sampled per horse
758 sampled per sheep
758 sampled per train

dropping duplicates based on filename

After sampling, number of positive labels per class as follows: aeroplane      758
bird           773
boat           843
chair          893
cow            804
diningtable    884
dog            867
horse          856
sheep          806
train          772
dtype: int64, number of records in df_sample = 7240
(100, 256, 256, 3)
(100, 10)
the shape of X_train is (70, 256, 256, 3)
the shape of X_val is (15, 256, 256, 3)
the shape of X_test is (15, 256, 256, 3)
the shape of y_train is (70, 10)
the shape of y_val is (15, 10)
the shape of y_test is (15, 10)


Make sure we're using GPU




In [18]:
model = load_model("../raw_data/models/resnet152")
model_name = "tuned_resnet152_existing_test"
model.summary()
    
print("\n\nIndex the number of layers in the ResNet152 base\n\n")
    
model.layers[0].summary()
print(f"There are {len(model.layers[0].layers)} layers in the base model.")

# freeze base, with exception of last layer
model.trainable = True
set_trainable = False

model.summary()
for layer in model.layers[0].layers:
    if 'conv5' in layer.name:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
    
# sanity check on trainable/untrainable params in base
model.layers[0].summary()
    
# sanity check on trainable/untrainable params in model
print(f"Model summary after unfreezing conv5_block3 for training")
model.summary()

resnet50_X_train = preproc_resnet(X_train) 
resnet50_X_val = preproc_resnet(X_val)
resnet50_X_test = preproc_resnet(X_test)
    
#     # Custom Scheduler Function
# #     lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,
# #                                                               decay_steps=10000,
# #                                                               decay_rate=0.9)
        
#     # compile and fit the model; during trainining, augment training images, change batch size to 16
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   optimizer=optimizers.Adam(learning_rate=1e-5),
                   metrics=['accuracy'])
            
es = EarlyStopping(monitor = 'val_accuracy', 
                    mode = 'max', 
                    patience = 30, 
                    verbose = 1, 
                    restore_best_weights = True)
    
# #    callbacks_list = [loss_history, lrate, es]
history = model.fit(resnet50_X_train, y_train, 
                    validation_data=(resnet50_X_val, y_val), 
                    epochs=150, 
                    batch_size=32, 
                    callbacks=[es])
    
# if os.path.exists(f"../raw_data/models/{model_name}/"):
#     model.save(f"../raw_data/models/{model_name}")
# else:
#     os.makedirs(f"../raw_data/models/{model_name}")
#     print(f"The directory for {model_name} is created.")
    
# model.save(f"../raw_data/models/{model_name}")
# print(f"Model {model_name} saved.")
# return model, model_name, history, resnet50_X_train, resnet50_X_val, resnet50_X_test

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 8, 8, 2048)        58370944  
                                                                 
 flatten_4 (Flatten)         (None, 131072)            0         
                                                                 
 dense_8 (Dense)             (None, 500)               65536500  
                                                                 
 dense_9 (Dense)             (None, 10)                5010      
                                                                 
Total params: 123,912,454
Trainable params: 65,541,510
Non-trainable params: 58,370,944
_________________________________________________________________


Index the number of layers in the ResNet152 base


Model: "resnet152"
____________________________________________________________________________________________

                                                                                                  
 conv2_block2_add (Add)         (None, 64, 64, 256)  0           ['conv2_block1_out[0][0]',       
                                                                  'conv2_block2_3_bn[0][0]']      
                                                                                                  
 conv2_block2_out (Activation)  (None, 64, 64, 256)  0           ['conv2_block2_add[0][0]']       
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 64, 64, 64)   16448       ['conv2_block2_out[0][0]']       
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 64, 64, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv3_block2_add (Add)         (None, 32, 32, 512)  0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_bn[0][0]']      
                                                                                                  
 conv3_block2_out (Activation)  (None, 32, 32, 512)  0           ['conv3_block2_add[0][0]']       
                                                                                                  
 conv3_block3_1_conv (Conv2D)   (None, 32, 32, 128)  65664       ['conv3_block2_out[0][0]']       
                                                                                                  
 conv3_block3_1_bn (BatchNormal  (None, 32, 32, 128)  512        ['conv3_block3_1_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv3_block6_1_conv (Conv2D)   (None, 32, 32, 128)  65664       ['conv3_block5_out[0][0]']       
                                                                                                  
 conv3_block6_1_bn (BatchNormal  (None, 32, 32, 128)  512        ['conv3_block6_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block6_1_relu (Activatio  (None, 32, 32, 128)  0          ['conv3_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block6_2_conv (Conv2D)   (None, 32, 32, 128)  147584      ['conv3_block6_1_relu[0][0]']    
          

                                                                                                  
 conv4_block1_1_relu (Activatio  (None, 16, 16, 256)  0          ['conv4_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block1_2_conv (Conv2D)   (None, 16, 16, 256)  590080      ['conv4_block1_1_relu[0][0]']    
                                                                                                  
 conv4_block1_2_bn (BatchNormal  (None, 16, 16, 256)  1024       ['conv4_block1_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block1_2_relu (Activatio  (None, 16, 16, 256)  0          ['conv4_block1_2_bn[0][0]']      
 n)       

...

 conv5_block2_add (Add)         (None, 8, 8, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_block2_out (Activation)  (None, 8, 8, 2048)   0           ['conv5_block2_add[0][0]']       
                                                                                                  
 conv5_block3_1_conv (Conv2D)   (None, 8, 8, 512)    1049088     ['conv5_block2_out[0][0]']       
                                                                                                  
 conv5_block3_1_bn (BatchNormal  (None, 8, 8, 512)   2048        ['conv5_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_blo

/home/erniesg/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2022-09-28 20:05:10.806589: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-09-28 20:05:11.813353: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-09-28 20:05:11.814136: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-09-28 20:05:1

3/3 [==============================] - ETA: 0s - loss: 0.3137 - accuracy: 0.6000

/home/erniesg/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


3/3 [==============================] - 12s 2s/step - loss: 0.3137 - accuracy: 0.6000 - val_loss: 0.0514 - val_accuracy: 0.8667
Epoch 2/150
3/3 [==============================] - 1s 269ms/step - loss: 0.0801 - accuracy: 0.8429 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 3/150
3/3 [==============================] - 0s 141ms/step - loss: 0.0085 - accuracy: 0.9857 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 4/150
3/3 [==============================] - 0s 142ms/step - loss: 0.0018 - accuracy: 0.9857 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 5/150
3/3 [==============================] - 0s 144ms/step - loss: 0.0011 - accuracy: 0.9857 - val_loss: 9.6381e-04 - val_accuracy: 1.0000
Epoch 6/150
3/3 [==============================] - 0s 142ms/step - loss: 7.4799e-04 - accuracy: 0.9857 - val_loss: 7.1529e-04 - val_accuracy: 1.0000
Epoch 7/150
3/3 [==============================] - 0s 141ms/step - loss: 5.6418e-04 - accuracy: 0.9857 - val_loss: 5.8280e-04 - val_accuracy: 1.0000
Epoch 8

In [11]:
for layer in model.layers:
    print(layer)

In [13]:
model.layers[0].layers

In [9]:
model, model_name, history, resnet50_X_train, resnet50_X_val, resnet50_X_test = tuner()

758 sampled per aeroplane
758 sampled per bird
758 sampled per boat
758 sampled per chair
758 sampled per cow
758 sampled per diningtable
758 sampled per dog
758 sampled per horse
758 sampled per sheep
758 sampled per train

dropping duplicates based on filename

After sampling, number of positive labels per class as follows: aeroplane      758
bird           775
boat           823
chair          869
cow            808
diningtable    895
dog            868
horse          857
sheep          794
train          770
dtype: int64, number of records in df_sample = 7237
(100, 256, 256, 3)
(100, 10)
the shape of X_train is (70, 256, 256, 3)
the shape of X_val is (15, 256, 256, 3)
the shape of X_test is (15, 256, 256, 3)
the shape of y_train is (70, 10)
the shape of y_val is (15, 10)
the shape of y_test is (15, 10)


Make sure we're using GPU


cuda_malloc_async
memory usage 0.495651024 GB


Fine-tune pre-trained ResNet152 model


Model: "sequential_4"
__________________________________________

 conv2_block2_2_bn (BatchNormal  (None, 64, 64, 64)  256         ['conv2_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block2_2_relu (Activatio  (None, 64, 64, 64)  0           ['conv2_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block2_3_conv (Conv2D)   (None, 64, 64, 256)  16640       ['conv2_block2_2_relu[0][0]']    
                                                                                                  
 conv2_block2_3_bn (BatchNormal  (None, 64, 64, 256)  1024       ['conv2_block2_3_conv[0][0]']    
 ization)                                                                                         
          

 conv3_block2_2_bn (BatchNormal  (None, 32, 32, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 32, 32, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 32, 32, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_3_bn (BatchNormal  (None, 32, 32, 512)  2048       ['conv3_block2_3_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv3_block5_3_conv (Conv2D)   (None, 32, 32, 512)  66048       ['conv3_block5_2_relu[0][0]']    
                                                                                                  
 conv3_block5_3_bn (BatchNormal  (None, 32, 32, 512)  2048       ['conv3_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block5_add (Add)         (None, 32, 32, 512)  0           ['conv3_block4_out[0][0]',       
                                                                  'conv3_block5_3_bn[0][0]']      
                                                                                                  
 conv3_block5_out (Activation)  (None, 32, 32, 512)  0           ['conv3_block5_add[0][0]']       
          

                                                                                                  
 conv3_block8_add (Add)         (None, 32, 32, 512)  0           ['conv3_block7_out[0][0]',       
                                                                  'conv3_block8_3_bn[0][0]']      
                                                                                                  
 conv3_block8_out (Activation)  (None, 32, 32, 512)  0           ['conv3_block8_add[0][0]']       
                                                                                                  
 conv4_block1_1_conv (Conv2D)   (None, 16, 16, 256)  131328      ['conv3_block8_out[0][0]']       
                                                                                                  
 conv4_block1_1_bn (BatchNormal  (None, 16, 16, 256)  1024       ['conv4_block1_1_conv[0][0]']    
 ization)                                                                                         
          

...

                                                                                                  
 conv4_block30_1_conv (Conv2D)  (None, 16, 16, 256)  262400      ['conv4_block29_out[0][0]']      
                                                                                                  
 conv4_block30_1_bn (BatchNorma  (None, 16, 16, 256)  1024       ['conv4_block30_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block30_1_relu (Activati  (None, 16, 16, 256)  0          ['conv4_block30_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block30_2_conv (Conv2D)  (None, 16, 16, 256)  590080      ['conv4_block30_1_relu[0][0]']   
          

                                )                                                                 
                                                                                                  
 conv4_block33_1_conv (Conv2D)  (None, 16, 16, 256)  262400      ['conv4_block32_out[0][0]']      
                                                                                                  
 conv4_block33_1_bn (BatchNorma  (None, 16, 16, 256)  1024       ['conv4_block33_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block33_1_relu (Activati  (None, 16, 16, 256)  0          ['conv4_block33_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_blo

 conv4_block35_out (Activation)  (None, 16, 16, 1024  0          ['conv4_block35_add[0][0]']      
                                )                                                                 
                                                                                                  
 conv4_block36_1_conv (Conv2D)  (None, 16, 16, 256)  262400      ['conv4_block35_out[0][0]']      
                                                                                                  
 conv4_block36_1_bn (BatchNorma  (None, 16, 16, 256)  1024       ['conv4_block36_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block36_1_relu (Activati  (None, 16, 16, 256)  0          ['conv4_block36_1_bn[0][0]']     
 on)                                                                                              
          

 conv5_block2_add (Add)         (None, 8, 8, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_block2_out (Activation)  (None, 8, 8, 2048)   0           ['conv5_block2_add[0][0]']       
                                                                                                  
 conv5_block3_1_conv (Conv2D)   (None, 8, 8, 512)    1049088     ['conv5_block2_out[0][0]']       
                                                                                                  
 conv5_block3_1_bn (BatchNormal  (None, 8, 8, 512)   2048        ['conv5_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_blo

NameError: name 'X_train' is not defined

In [19]:
def evaluate(model, model_name, X_test, y_test):
    res = model.evaluate(X_test, y_test)
    res
    test_accuracy = res[-1]
    print(f"test_accuracy = {round(test_accuracy,2)*100} %")
        
    pd.DataFrame(res).to_csv(f"../raw_data/models/{model_name}/res.csv")

    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('../raw_data/models/' + str(model_name) + '/' + 'accuracy.png')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('../raw_data/models/' + str(model_name) + '/' + 'loss.png')
    plt.show()
    y_pred = model.predict(X_test)
    return model, y_pred

In [20]:
model, y_pred = evaluate(model, model_name, resnet50_X_test, y_test)
trainer.generate_reports(model, model_name, y_pred)

1/1 [==============================] - 0s 395ms/step - loss: 0.0078 - accuracy: 1.0000
test_accuracy = 100.0 %


OSError: Cannot save file into a non-existent directory: '../raw_data/models/tuned_resnet152_existing_test'